In [57]:

import pandas as pd
import os
import pandas_ta as ta  
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, 
                        message='DataFrameGroupBy.apply operated on the grouping columns')

RAW_DIR = "../data/raw/yfinance"
PROCESSED_DIR = "../data/processed"
print("Files in yfinance folder:", os.listdir(RAW_DIR))


Files in yfinance folder: ['.gitkeep', 'AAPL.csv', 'ABNB.csv', 'ADBE.csv', 'ADI.csv', 'ADP.csv', 'ADSK.csv', 'AEP.csv', 'AMD.csv', 'AMGN.csv', 'AMZN.csv', 'ANSS.csv', 'ASML.csv', 'AVGO.csv', 'AZN.csv', 'BIIB.csv', 'BKNG.csv', 'BKR.csv', 'CDNS.csv', 'CEG.csv', 'CHTR.csv', 'CMCSA.csv', 'COST.csv', 'CPRT.csv', 'CRWD.csv', 'CSCO.csv', 'CSX.csv', 'CTAS.csv', 'DASH.csv', 'DDOG.csv', 'DLTR.csv', 'DXCM.csv', 'EA.csv', 'ENPH.csv', 'EXC.csv', 'FANG.csv', 'FAST.csv', 'FTNT.csv', 'GEHC.csv', 'GFS.csv', 'GILD.csv', 'GOOG.csv', 'GOOGL.csv', 'HES.csv', 'HON.csv', 'IDXX.csv', 'ILMN.csv', 'INTC.csv', 'INTU.csv', 'ISRG.csv', 'JD.csv', 'KDP.csv', 'KHC.csv', 'KLAC.csv', 'LCID.csv', 'LRCX.csv', 'LULU.csv', 'MAR.csv', 'MCHP.csv', 'MDLZ.csv', 'MELI.csv', 'META.csv', 'MNST.csv', 'MRNA.csv', 'MRVL.csv', 'MSFT.csv', 'MU.csv', 'NFLX.csv', 'NVDA.csv', 'NXPI.csv', 'ODFL.csv', 'ON.csv', 'ORLY.csv', 'PANW.csv', 'PAYX.csv', 'PCAR.csv', 'PDD.csv', 'PEP.csv', 'PYPL.csv', 'QCOM.csv', 'REGN.csv', 'RIVN.csv', 'ROST.csv', 

Bulk Load All YFinance Files


In [58]:
from tqdm import tqdm   
def load_all_yfinance():
    """Load and combine multiple YFinance stock data CSVs into one DataFrame.
    Handles metadata rows, standardizes columns, and ensures date continuity."""
    
    all_dfs = []  
    
    for file in tqdm(os.listdir(RAW_DIR)):
        if file.endswith('.csv'):
            try:
                ticker = file.split('.')[0]
                
                # Read CSV with specific formatting:
                # - Skip 3 metadata rows
                # - Standardize column names
                # - Parse dates as datetime objects
                df = pd.read_csv(
                    f"{RAW_DIR}/{file}",
                    skiprows=3,
                    names=['date', 'close', 'high', 'low', 'open', 'volume'],
                    parse_dates=['date']
                )
                
                df['ticker'] = ticker
                
                # Ensure continuous daily data:
                # 1. Set date as index
                # 2. Force daily frequency (inserts NaNs for missing dates)
                # 3. Forward-fill missing values
                # 4. Reset index to return date to a column
                df = df.set_index('date').asfreq('D').ffill().reset_index()
                
                all_dfs.append(df)
                
            except Exception as e:
                print(f"⚠️ Failed {file}: {str(e)}")
                continue
                
    # Combine all DataFrames, ignoring original indices
    return pd.concat(all_dfs, ignore_index=True)

# Execute and verify load
df = load_all_yfinance()
print(f"✅ Loaded {len(df)} rows from {df['ticker'].nunique()} stocks")

100%|██████████| 101/101 [00:00<00:00, 285.50it/s]

✅ Loaded 36200 rows from 100 stocks


code verification 

In [59]:
print(df[df['ticker'] == 'AAPL'].head(10))
unique_tickers = df['ticker'].unique().tolist()
print(f"Tickers: {unique_tickers[:5]}...")  

        date       close        high         low        open      volume  \
0 2024-04-01  169.230927  170.445194  168.683508  170.385479  46240500.0   
1 2024-04-02  168.046509  168.544159  167.439375  168.285386  49329500.0   
2 2024-04-03  168.852692  169.877850  167.787728  167.996733  47691700.0   
3 2024-04-04  168.026627  171.112049  168.026627  169.489704  53704400.0   
4 2024-04-05  168.783020  169.589211  168.155976  168.792968  42055200.0   
5 2024-04-06  168.783020  169.589211  168.155976  168.792968  42055200.0   
6 2024-04-07  168.783020  169.589211  168.155976  168.792968  42055200.0   
7 2024-04-08  167.658340  168.404816  167.449336  168.235616  37425500.0   
8 2024-04-09  168.872604  169.280681  167.558816  167.907162  42451200.0   
9 2024-04-10  166.991486  168.295327  166.324636  168.006696  49709300.0   

  ticker  
0   AAPL  
1   AAPL  
2   AAPL  
3   AAPL  
4   AAPL  
5   AAPL  
6   AAPL  
7   AAPL  
8   AAPL  
9   AAPL  
Tickers: ['AAPL', 'ABNB', 'ADBE', 'ADI', '

Data Cleaning Pipeline
Goal: Fix common data quality issues

Why?
Ensures dates are recognized as timestamps (not strings)
, Handles market closures without leaving gaps
, Guarantees no NaN values break your models



In [60]:
def clean_data(df):
    df['date'] = pd.to_datetime(df['date'])  
    
    df = df.sort_values(['ticker', 'date'])
    df = df.groupby('ticker').apply(lambda x: x.ffill())  
    
    return df.dropna()

In [61]:
total_nans = df.isna().sum().sum()
print(f"Total NaN values: {total_nans}")

nan_per_column = df.isna().sum()
print("\nNaN per column:")
print(nan_per_column)
print(df.shape)
print(df.head(10))

Total NaN values: 0

NaN per column:
date      0
close     0
high      0
low       0
open      0
volume    0
ticker    0
dtype: int64
(36200, 7)
        date       close        high         low        open      volume  \
0 2024-04-01  169.230927  170.445194  168.683508  170.385479  46240500.0   
1 2024-04-02  168.046509  168.544159  167.439375  168.285386  49329500.0   
2 2024-04-03  168.852692  169.877850  167.787728  167.996733  47691700.0   
3 2024-04-04  168.026627  171.112049  168.026627  169.489704  53704400.0   
4 2024-04-05  168.783020  169.589211  168.155976  168.792968  42055200.0   
5 2024-04-06  168.783020  169.589211  168.155976  168.792968  42055200.0   
6 2024-04-07  168.783020  169.589211  168.155976  168.792968  42055200.0   
7 2024-04-08  167.658340  168.404816  167.449336  168.235616  37425500.0   
8 2024-04-09  168.872604  169.280681  167.558816  167.907162  42451200.0   
9 2024-04-10  166.991486  168.295327  166.324636  168.006696  49709300.0   

  ticker  
0   AAP

Feature Engineering
Goal: Add technical indicators traders use

Why These Indicators?

SMA: Smooths price noise to reveal trends

RSI: Identifies potential reversals (values >70 = overbought, <30 = oversold)

MACD: Shows momentum shifts

Bollinger Bands: Highlights volatility extremes

In [62]:
def add_technical_indicators(df):
    """Calculate technical indicators for each stock in the DataFrame.
    
    Applies common trading indicators to each stock's price series independently
    to avoid cross-contamination between different securities.
    
    Args:
        df: DataFrame containing stock data with columns: ['ticker', 'date', 'close', ...]
        
    Returns:
        DataFrame with additional technical indicator columns:
        - sma_20: 20-day Simple Moving Average (trend identification)
        - rsi_14: 14-day Relative Strength Index (momentum)
        - macd: MACD line (12,26,9 EMAs crossover system)
        - boll_high: Upper Bollinger Band (2 std dev above SMA)
        - boll_low: Lower Bollinger Band (2 std dev below SMA)
    """
    
    # Group by ticker to ensure calculations are performed per-stock
    # Using apply(lambda) pattern for clean column-wise operations
    return df.groupby('ticker').apply(lambda x: x.assign(
        # Trend indicator: 20-period Simple Moving Average
        # Helps identify the prevailing market direction
        sma_20=ta.sma(x['close'], 20),
        
        # Momentum oscillator: 14-period RSI
        # Measures speed of price movements (range: 0-100)
        # >70 = overbought, <30 = oversold
        rsi_14=ta.rsi(x['close'], 14),
        
        # MACD Line (12,26,9 EMA configuration)
        # Shows relationship between two moving averages
        # Positive = upward momentum, Negative = downward momentum
        macd=ta.macd(x['close'])['MACD_12_26_9'],
        
        # Bollinger Bands (20MA ± 2 standard deviations)
        # Identifies overextended price moves
        # Prices near upper band = potentially overbought
        # Prices near lower band = potentially oversold
        boll_high=ta.bbands(x['close'])['BBU_5_2.0'],
        boll_low=ta.bbands(x['close'])['BBL_5_2.0']
    ))

In [63]:
df_features = add_technical_indicators(df)
print("New columns:", df_features.columns.tolist())
print("\nAAPL data with indicators:")
print(df_features[df_features['ticker'] == 'AAPL'].tail(3))

New columns: ['date', 'close', 'high', 'low', 'open', 'volume', 'ticker', 'sma_20', 'rsi_14', 'macd', 'boll_high', 'boll_low']

AAPL data with indicators:
                 date       close        high         low        open  \
ticker                                                                  
AAPL   359 2025-03-26  221.529999  225.020004  220.470001  223.509995   
       360 2025-03-27  223.850006  224.990005  220.559998  221.389999   
       361 2025-03-28  217.899994  223.809998  217.679993  221.669998   

                volume ticker      sma_20     rsi_14      macd   boll_high  \
ticker                                                                       
AAPL   359  34466100.0   AAPL  220.475502  45.985743 -3.742026  224.668996   
       360  37094800.0   AAPL  219.714502  50.019522 -3.144314  225.776406   
       361  39784100.0   AAPL  218.656001  41.466587 -3.114834  225.945208   

              boll_low  
ticker                  
AAPL   359  216.351005  
       360  2

In [64]:
# Forward-fill missing values (carry last known observation forward)
# NOTE: This maintains continuous time series but WON'T fill:
# 1. Leading NaNs (where no prior value exists to fill from)
# 2. Technical indicator warm-up periods (e.g., first 19 rows of SMA20)
# For complete NaN removal, use .dropna() after this
df_features = df_features.ffill()

# drop early rows with NaN
df_features = df_features.dropna()

print("Remaining NaNs:", df_features.isna().sum().sum())  # Should be 0
print(df_features.head())

Remaining NaNs: 0
                date       close        high         low        open  \
ticker                                                                 
AAPL   25 2024-04-26  168.504349  170.534755  168.384902  169.081625   
       26 2024-04-27  168.504349  170.534755  168.384902  169.081625   
       27 2024-04-28  168.504349  170.534755  168.384902  169.081625   
       28 2024-04-29  172.684601  175.202709  172.286487  172.555207   
       29 2024-04-30  169.529495  174.167598  169.201044  172.515396   

               volume ticker      sma_20     rsi_14      macd   boll_high  \
ticker                                                                      
AAPL   25  44838400.0   AAPL  168.864148  50.389412 -1.162208  170.483094   
       26  44838400.0   AAPL  168.850214  50.389412 -1.007623  170.140261   
       27  44838400.0   AAPL  168.892515  50.389412 -0.875026  169.134192   
       28  68169400.0   AAPL  169.083115  63.084174 -0.427701  172.716504   
       29  6593

Check if any ticker has date gaps


In [65]:
date_gaps = df.groupby('ticker')['date'].apply(lambda x: x.sort_values().diff().max())
print("Max gap between dates per ticker (should be <=3 days for weekends):")
print(date_gaps.dt.days.value_counts())

Max gap between dates per ticker (should be <=3 days for weekends):
date
1    100
Name: count, dtype: int64


Check for duplicate dates per ticker


In [66]:
duplicate_dates = df.groupby(['ticker', 'date']).size().loc[lambda x: x > 1]
if not duplicate_dates.empty:
    print("⚠️ Duplicate dates found:", duplicate_dates.index.tolist())

# Verify volume > 0 for all rows
zero_volume = df[df['volume'] <= 0]
if not zero_volume.empty:
    print("⚠️ Stocks with zero volume:", zero_volume['ticker'].unique())

Final Data Quality Checks

In [67]:
# Key Tasks:
# 1. Resolve data structure issues
# 2. Validate data quality
# 3. Calculate technical indicators
# 4. Handle missing data appropriately

# 1. CREATE CLEAN WORKING COPY
# Always work on copies to prevent accidental mutation of source data
# Note: This breaks reference chains that can cause SettingWithCopy warnings
df_features = df_features.copy()

# 2. FIX INDEX/COLUMN CONFLICTS
# Common issue after groupby operations - ticker ends up in both index and columns
if 'ticker' in df_features.index.names:
    # Save the index values before resetting
    ticker_values = df_features.index.get_level_values('ticker')
    
    # Clean slate - remove all index levels
    # drop=True prevents old index from becoming new column
    df_features = df_features.reset_index(drop=True)
    
    # Only add ticker back if it's not already a column
    # Prevents duplicate columns which break pandas operations
    if 'ticker' not in df_features.columns:
        df_features['ticker'] = ticker_values

# 3. DATA QUALITY CHECKS
print("\n🔍 Running Data Quality Checks...")

# Check 1: Negative Prices (impossible in real markets)
# Important because some indicators break with negative values
negative_prices = df_features[
    (df_features['close'] <= 0) | 
    (df_features['open'] <= 0) |
    (df_features['high'] <= 0) |
    (df_features['low'] <= 0)
]
if not negative_prices.empty:
    print("⚠️ Negative prices found in:", negative_prices['ticker'].unique())
    # Remove corrupt rows completely - can't fix bad prices
    df_features = df_features[~df_features.index.isin(negative_prices.index)]

# Check 2: Price Relationship Sanity
# Ensures high > low, high >= close, etc. (market mechanics)
invalid_prices = df_features[
    (df_features['high'] < df_features['low']) |
    (df_features['high'] < df_features['close']) |
    (df_features['low'] > df_features['open'])
]
if not invalid_prices.empty:
    print("⚠️ Illogical prices in:", invalid_prices['ticker'].unique())
    # Drop invalid rows - likely data corruption
    df_features = df_features[~df_features.index.isin(invalid_prices.index)]

# 4. FEATURE ENGINEERING (WITH NaN PROTECTION)
print("\n⚙️ Calculating Features with Safe NaN Handling...")

# Volume Analysis - helps detect unusual activity
# Using min_periods=1 means we get values even with limited history
df_features['volume_ma_20'] = df_features.groupby('ticker')['volume'].transform(
    lambda x: x.rolling(20, min_periods=1).mean()
)
# Volume spike ratio (safe division)
df_features['volume_spike'] = np.where(
    df_features['volume_ma_20'] > 0,  # Prevent divide-by-zero
    df_features['volume'] / df_features['volume_ma_20'],
    1.0  # Neutral value when no history exists
)

# Technical Indicators Wrapper
# Group-level calculation with error handling
def safe_technical(group):
    """Calculate indicators for a single stock with error protection"""
    try:
        # Trend indicators
        group['sma_20'] = ta.sma(group['close'], length=20)  # 20-day moving average
        # Momentum indicators
        group['rsi_14'] = ta.rsi(group['close'], length=14)  # Relative Strength Index
        # Volatility indicator
        group['atr_14'] = ta.atr(group['high'], group['low'], group['close'], length=14)
        return group
    except Exception as e:
        # Log errors but keep processing other stocks
        print(f"⚠️ Error calculating indicators for {group.name}: {str(e)}")
        return group

# Apply to all stocks in parallel
df_features = df_features.groupby('ticker', group_keys=False).apply(safe_technical)

# 5. NaN MANAGEMENT STRATEGY
print("\n🧹 Final NaN Cleanup...")

# Technical indicators create NaNs during warm-up periods:
# - SMA20: First 19 days
# - RSI14: First 13 days  
# Strategy: Forward fill within each stock's data
tech_cols = ['sma_20', 'rsi_14', 'atr_14']
for col in tech_cols:
    if col in df_features.columns:
        # Fill NaNs with last valid observation per stock
        df_features[col] = df_features.groupby('ticker')[col].ffill()

# Final NaN treatment - domain-specific fallbacks
fill_values = {
    'sma_20': df_features['close'],  # If no SMA, use raw price
    'rsi_14': 50,                   # Neutral RSI level
    'atr_14': 0,                     # Assume no volatility
    'volume_spike': 1                # No spike
}
for col, val in fill_values.items():
    if col in df_features.columns:
        df_features[col] = df_features[col].fillna(val)

# 6. FINAL VALIDATION
print("\n✅ Final Validation")
# Hard checks - fail fast if data isn't clean
assert df_features.isna().sum().sum() == 0, (
    f"Final NaN count: {df_features.isna().sum().sum()}\n"
    f"NaN columns: {df_features.columns[df_features.isna().any()].tolist()}"
)
assert (df_features['close'] > 0).all(), "Negative prices exist!"

# Ensure chronological order for time series analysis
df_features.sort_values(['ticker', 'date'], inplace=True)

print(f"\n🎉 Cleaning complete! Final shape: {df_features.shape}")
print("Available columns:", df_features.columns.tolist())


🔍 Running Data Quality Checks...

⚙️ Calculating Features with Safe NaN Handling...

🧹 Final NaN Cleanup...

✅ Final Validation

🎉 Cleaning complete! Final shape: (36175, 15)
Available columns: ['date', 'close', 'high', 'low', 'open', 'volume', 'ticker', 'sma_20', 'rsi_14', 'macd', 'boll_high', 'boll_low', 'volume_ma_20', 'volume_spike', 'atr_14']


MARKET DAY FLAG


In [68]:
# ------------------
# Creates boolean flag for trading vs non-trading days
# Critical for: 
# - Avoiding false signals from weekend/holiday data
# - Volume spike analysis
# Note: Uses volume > 0 as proxy for market open (more reliable than calendar)
df_features['is_market_open'] = df_features['volume'] > 0

# PRICE CHANGE ANALYSIS
# ------------------------
# Calculates daily returns with protection against:
# - Division by zero
# - Look-ahead bias
# Groups by ticker to prevent cross-stock contamination
df_features['daily_return'] = df_features.groupby('ticker')['close'].pct_change()

# Extreme move detection (50% daily change)
# Important for:
# - Identifying potential data errors
# - Flagging corporate actions (splits, dividends)
# - Risk management scenarios
extreme_returns = df_features[np.abs(df_features['daily_return']) > 0.5]
if not extreme_returns.empty:
    # Log but don't fail - some stocks legitimately gap
    print(f"⚠️ {len(extreme_returns)} extreme returns (>50%) detected")
    print("Sample affected tickers:", extreme_returns['ticker'].unique()[:5])
    


print(df_features.head(10))

# DATA PERSISTENCE
# -------------------
# Saves cleaned data in Parquet format because:
# - Preserves dtypes (no float→string conversion like CSV)
# - Efficient columnar storage (faster for time series)
# - Built-in compression (gzip offers good balance)
# - Maintains schema on read
df_features.to_parquet(
    f"{PROCESSED_DIR}/cleaned_stocks.parquet",  # Versioned path recommended
    index=False,  # Don't persist index (redundant with ticker+date)
    engine='pyarrow',  # More reliable than fastparquet
    compression='gzip',  # ~60-70% size reduction
)

# Post-save verification
saved_size = os.path.getsize(f"{PROCESSED_DIR}/cleaned_stocks.parquet")/1e6
print(f"✅ Saved cleaned data ({saved_size:.1f}MB)")
print(f"Columns persisted: {df_features.columns.tolist()}")

        date       close        high         low        open       volume  \
0 2024-04-26  168.504349  170.534755  168.384902  169.081625   44838400.0   
1 2024-04-27  168.504349  170.534755  168.384902  169.081625   44838400.0   
2 2024-04-28  168.504349  170.534755  168.384902  169.081625   44838400.0   
3 2024-04-29  172.684601  175.202709  172.286487  172.555207   68169400.0   
4 2024-04-30  169.529495  174.167598  169.201044  172.515396   65934800.0   
5 2024-05-01  168.504349  171.898327  168.315239  168.783032   50383100.0   
6 2024-05-02  172.216827  172.604994  170.086885  171.699267   94214900.0   
7 2024-05-03  182.518188  186.121171  181.801571  185.772810  163224100.0   
8 2024-05-04  182.518188  186.121171  181.801571  185.772810  163224100.0   
9 2024-05-05  182.518188  186.121171  181.801571  185.772810  163224100.0   

  ticker      sma_20  rsi_14      macd   boll_high    boll_low  volume_ma_20  \
0   AAPL  168.504349    50.0 -1.162208  170.483094  164.316027  4.483840

In [69]:
df_features.head()

,date,close,high,low,open,volume,ticker,sma_20,rsi_14,macd,boll_high,boll_low,volume_ma_20,volume_spike,atr_14,is_market_open,daily_return
0,2024-04-26,168.504349,170.534755,168.384902,169.081625,44838400.0,AAPL,168.504349,50.0,-1.162208,170.483094,164.316027,44838400.0,1.00000,0.0,True,NaN
1,2024-04-27,168.504349,170.534755,168.384902,169.081625,44838400.0,AAPL,168.504349,50.0,-1.007623,170.140261,166.036362,44838400.0,1.00000,0.0,True,0.000000
2,2024-04-28,168.504349,170.534755,168.384902,169.081625,44838400.0,AAPL,168.504349,50.0,-0.875026,169.134192,167.997924,44838400.0,1.00000,0.0,True,0.000000
3,2024-04-29,172.684601,175.202709,172.286487,172.555207,68169400.0,AAPL,172.684601,50.0,-0.427701,172.716504,166.199188,50671150.0,1.34533,0.0,True,0.024808
4,2024-04-30,169.529495,174.167598,169.201044,172.515396,65934800.0,AAPL,169.529495,50.0,-0.324047,172.783477,166.307380,53723880.0,1.22729,0.0,True,-0.018271
